**DS105M 2022 Week09**

**Topic:** Managing your Data Science Workflow

**Author:** [@jonjoncardoso](github.com/jonjoncardoso)

**Date:** 22 November 2022

---

# What is in my `conda`?

In [52]:
!conda list

# packages in environment at C:\Users\jonat\anaconda3\envs\.venv:
#
# Name                    Version                   Build  Channel
anyio                     3.5.0           py310haa95532_0  
argon2-cffi               21.3.0             pyhd3eb1b0_0  
argon2-cffi-bindings      21.2.0          py310h2bbff1b_0  
asttokens                 2.0.5              pyhd3eb1b0_0  
atomicwrites              1.4.1                    pypi_0    pypi
attrs                     21.4.0             pyhd3eb1b0_0  
babel                     2.9.1              pyhd3eb1b0_0  
backcall                  0.2.0              pyhd3eb1b0_0  
beautifulsoup4            4.11.1          py310haa95532_0  
blas                      1.0                         mkl  
bleach                    4.1.0              pyhd3eb1b0_0  
blosc                     1.21.1               h74325e0_3    conda-forge
boost-cpp                 1.78.0               h9f4b32c_1    conda-forge
bottleneck                1.3.5           py310h91289

Let's save the content of this file to a conda_list text file. Replace my username for yours!

In [53]:
!conda list >> conda_list_jonjoncardoso.txt

Where are we, again?

In [26]:
!dir

 Volume in drive C is OS
 Volume Serial Number is 0890-5E2A

 Directory of C:\Users\jonat\Workspace\lse-ds105-course-notes\resources\week09

22/11/2022  12:24    <DIR>          .
22/11/2022  12:17    <DIR>          ..
22/11/2022  12:23    <DIR>          .ipynb_checkpoints
22/11/2022  12:20            16,626 conda_list.tsv
22/11/2022  12:23            16,626 conda_list.txt
22/11/2022  12:24             5,081 DS105M 2022 Week09.ipynb
               3 File(s)         38,333 bytes
               3 Dir(s)  642,999,566,336 bytes free


# Code to read the conda_list_<>.txt file

In [1]:
import pandas as pd

In [22]:
def parse_condalist(filename):
    """
    A function to parse the conda_list_<>.txt file
    """

    username = filename.split(".txt")[0].split("conda_list_")[1]
    df_condalist = pd.DataFrame()
    
    try:


        def parse_line(line):
            data = line.split()
            if len(data) == 3:
                return pd.DataFrame({"Name": data[0], "Version": data[1], "Build": data[2]}, index=[0])
            else:
                return pd.DataFrame({"Name": data[0], "Version": data[1], "Build": data[2], "Channel": data[3]}, index=[0])

        with open(filename, mode='r') as f:
            df_condalist = pd.concat([parse_line(line) for line in f.readlines() if not line.startswith("#")])

        df_condalist["Username"] = username
        df_condalist.reset_index(drop=True)

    except Exception as e:
        print("Error parsing: " + username)
        
    return df_condalist

**Read everyone's files**

In [8]:
import os
# os.listdir()

In [23]:
filelist = ['conda_list_adrienjoly.txt',
 'conda_list_hassam.txt',
 'conda_list_idil2.txt',
 'conda_list_inesmccairley.txt',
 'conda_list_Jessy_yang.txt',
 'conda_list_jonjoncardoso.txt',
 'conda_list_maia.txt',
 'conda_list_michaelhao.txt',
 'conda_list_sara.txt',
 'conda_list_suosi.txt',
 'conda_list_xf.txt']
 #'pip_list_tombaldwin.txt']

df = pd.concat([parse_condalist(filename) for filename in filelist])
df = df.set_index(["Username", "Name"])
df.head()

Error parsing: hassam
Error parsing: inesmccairley
Error parsing: xf


Version           Build Channel
Username   Name                                                   
adrienjoly _ipyw_jlab_nb_ext_conf    0.1.0  py39haa95532_0     NaN
           alabaster                0.7.12    pyhd3eb1b0_0     NaN
           anaconda                2022.10          py39_0     NaN
           anaconda-client          1.11.0  py39haa95532_0     NaN
           anaconda-navigator        2.3.1  py39haa95532_0     NaN

In [25]:
df.reset_index()["Username"].nunique()

8

**Do our versions of `pandas` match?**

In [29]:
df.query("Name == 'jupyterlab'").reset_index().set_index(["Name", "Version", "Username"])

Build Channel
Name       Version Username                             
jupyterlab 3.4.4   adrienjoly     py39haa95532_0     NaN
                   idil2          py39hecd8cb5_0     NaN
                   Jessy_yang     py39hecd8cb5_0     NaN
                   Jessy_yang     py39hecd8cb5_0     NaN
           3.4.2   jonjoncardoso          pypi_0    pypi
           3.5.0   maia                   pypi_0    pypi
           3.3.2   michaelhao       pyhd3eb1b0_0     NaN
           3.0.14  sara             pyhd3eb1b0_1     NaN
           3.4.4   suosi          py39hecd8cb5_0     NaN

**Where did we install packages from?**

In [33]:
df_summary = (
    df.groupby(["Channel", "Username"], dropna=False)
      .apply(lambda x: pd.Series({"count": len(x)}))
      .reset_index()
      .pivot_table(values="count", columns="Channel", index=["Username"], dropna=False, aggfunc='sum')
)

df_summary

Channel,conda-forge,pypi
Username,,
adrienjoly,NaN,4.0
jonjoncardoso,102.0,33.0
maia,NaN,93.0
michaelhao,NaN,4.0
suosi,NaN,8.0


**What are the most popular packages?** 

In [35]:
df.reset_index().groupby(["Name"]).apply(lambda x: pd.Series({"count_distinct": x["Username"].nunique()})).sort_values("count_distinct", ascending=False).head(15)

,count_distinct
Name,
statsmodels,8
terminado,8
parso,8
pandocfilters,8
sniffio,8
pandas,8
packaging,8
openssl,8
soupsieve,8
